In [6]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.set_option('display.max_columns', None)

In [7]:
import os
print(os.getcwd())

C:\PyCharm\PyCharm_Projects\century-health-pipeline\notebooks


## Data import

In [9]:
symptoms_df = pd.read_csv('../data/01_raw/symptoms.csv', delimiter=',', header=0)
patients_df = pd.read_csv('../data/01_raw/patients.csv', delimiter=',', header=0)
medications_df = pd.read_csv('../data/01_raw/medications.csv', delimiter=',', header=0)

encounters_df = pd.read_parquet('../data/01_raw/encounters.parquet', engine='pyarrow')

conditions_df = pd.read_excel('../data/01_raw/conditions.xlsx', engine='openpyxl', header=0)


## 1. Patient data Assessment

In [10]:
# High-level view
patients_df.head(4)

,PATIENT_ID,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,54f1059e-6250-3949-6dd0-1dda9b85d22a,2002-01-21,NaN,999-28-3364,S99987398,X44428214X,Ms.,Fredricka415,Crist667,NaN,NaN,NaN,white,nonhispanic,NaN,San Antonio Texas US,931 Bartell Ville Apt 84,Fort Worth,Texas,Tarrant County,48439.0,76104,0.329154,-97.399553,9990.08,0.0,35460
1,92675303-ca5b-136a-169b-e764c5753f06,1997-04-16,NaN,999-62-9859,S99999770,X75016560X,Mr.,Lorenzo669,Urrutia540,NaN,NaN,NaN,white,hispanic,NaN,Carolina Puerto Rico PR,386 Altenwerth Orchard Apt 67,Arlington,Texas,Tarrant County,48439.0,76006,0.328145,-97.068885,10936.80,0.0,88407
2,a0b63e97-b6fd-5fe1-8f2d-2bec915efa97,1993-02-04,NaN,999-53-6488,S99979170,X60079936X,Mr.,Luther918,MacGyver246,NaN,NaN,M,white,nonhispanic,NaN,Grand Prairie Texas US,250 Morar Fork Apt 66,Houston,Texas,Harris County,48339.0,77357,0.300834,-95.649706,10662.16,0.0,41800
3,abc59f62-dc5a-5095-1141-80b4ee8be73b,1995-05-23,NaN,999-37-1058,S99981031,X14759314X,Mrs.,Jacque955,Jones311,NaN,Will178,M,white,nonhispanic,NaN,Euless Texas US,492 Keebler Estate,Waco,Texas,McLennan County,48309.0,76655,0.315173,-97.292558,11455.93,0.0,41915


In [11]:
# Assessing data types and null-value counts 
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PATIENT_ID           10000 non-null  object 
 1   BIRTHDATE            10000 non-null  object 
 2   DEATHDATE            0 non-null      float64
 3   SSN                  10000 non-null  object 
 4   DRIVERS              7680 non-null   object 
 5   PASSPORT             7039 non-null   object 
 6   PREFIX               7337 non-null   object 
 7   FIRST                10000 non-null  object 
 8   LAST                 10000 non-null  object 
 9   SUFFIX               59 non-null     object 
 10  MAIDEN               2352 non-null   object 
 11  MARITAL              5854 non-null   object 
 12  RACE                 10000 non-null  object 
 13  ETHNICITY            10000 non-null  object 
 14  GENDER               0 non-null      float64
 15  BIRTHPLACE           10000 non-null  

In [12]:
# Evaluating outliers and data distribution of fields
patients_df.describe()

,DEATHDATE,GENDER,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
count,0.0,0.0,9710.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,NaN,NaN,48188.824717,74947.406200,0.309868,-97.548956,10489.406391,1817.903496,85358.40780
std,NaN,NaN,690.565166,13036.344689,0.020044,2.438536,5256.509164,2462.371193,141329.22638
min,NaN,NaN,35013.000000,0.000000,0.258876,-106.646755,100.000000,0.000000,-986794.00000
25%,NaN,NaN,48113.000000,75904.000000,0.296555,-98.195686,6889.780000,0.000000,25129.25000
50%,NaN,NaN,48201.000000,77099.000000,0.306608,-97.126683,10761.620000,0.000000,51009.50000
75%,NaN,NaN,48355.000000,78260.000000,0.327973,-95.921881,13374.712500,3578.720000,94730.75000
max,NaN,NaN,48507.000000,88063.000000,0.365969,-93.752308,27627.160000,10808.450000,998831.00000


In [13]:
# Checking for duplicate patient records

id_count = patients_df.groupby('PATIENT_ID')['PATIENT_ID'].count().reset_index(name='ID_count')
id_count['ID_count'].value_counts()


ID_count
1    10000
Name: count, dtype: int64

In [14]:
# Assessing distribution of data where 'INCOME' is negative
patients_df[patients_df['INCOME']<0]['INCOME'].describe()

count       200.000000
mean    -106278.060000
std      157876.627054
min     -986794.000000
25%     -116196.250000
50%      -58557.000000
75%      -29461.750000
max        -383.000000
Name: INCOME, dtype: float64

In [15]:
# 'MARITAL' field value check
patients_df['MARITAL'].value_counts()

MARITAL
M    3643
S    1202
D     798
W     211
Name: count, dtype: int64

In [16]:
# 'PREFIX' field value check
patients_df['PREFIX'].value_counts()

PREFIX
Mr.     3665
Mrs.    2352
Ms.     1320
Name: count, dtype: int64

In [17]:
# 'STATE' field value check
patients_df['STATE'].value_counts()

STATE
Texas        9950
Louisiana      50
Name: count, dtype: int64

In [18]:
patients_df[(patients_df['MARITAL'].isna()) & (~patients_df['PREFIX'].isna())].head(4)

,PATIENT_ID,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,54f1059e-6250-3949-6dd0-1dda9b85d22a,2002-01-21,NaN,999-28-3364,S99987398,X44428214X,Ms.,Fredricka415,Crist667,NaN,NaN,NaN,white,nonhispanic,NaN,San Antonio Texas US,931 Bartell Ville Apt 84,Fort Worth,Texas,Tarrant County,48439.0,76104,0.329154,-97.399553,9990.08,0.0,35460
1,92675303-ca5b-136a-169b-e764c5753f06,1997-04-16,NaN,999-62-9859,S99999770,X75016560X,Mr.,Lorenzo669,Urrutia540,NaN,NaN,NaN,white,hispanic,NaN,Carolina Puerto Rico PR,386 Altenwerth Orchard Apt 67,Arlington,Texas,Tarrant County,48439.0,76006,0.328145,-97.068885,10936.80,0.0,88407
14,82340b68-7f78-8d50-15ba-0396be76a381,1998-11-16,NaN,999-82-6124,S99998080,X10220313X,Mr.,Zackary401,Lehner980,NaN,NaN,NaN,white,nonhispanic,NaN,Grapevine Texas US,427 Lemke Hollow,Austin,Texas,Travis County,48453.0,78733,0.302038,-97.715959,11436.34,0.0,68505
18,a832f5fa-07a9-e8ef-dc1a-8df6376be9cf,2003-06-22,NaN,999-53-3586,S99964102,X4379513X,Mr.,Korey682,Howell947,NaN,NaN,NaN,black,nonhispanic,NaN,San Antonio Texas US,542 Connelly Dam Suite 86,San Antonio,Texas,Bexar County,48029.0,78250,0.293931,-98.574212,10761.62,0.0,66991


## 2. Encounter data Assessment

In [19]:
# High-level view
encounters_df.head(4)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,6539c675-dcf2-cace-a90d-7f097d1799b3,2014-02-10T11:00:08Z,2014-02-10T11:15:08Z,54F1059E-6250-3949-6DD0-1DDA9B85D22A,ef42ec52-a1c1-3fc5-8e52-da46f7f3970c,35088bec-f756-35d8-8140-b2b017c0eacc,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,wellness,410620009,Well child visit (procedure),133.36,405.36,0.0,NaN,None
1,748a443c-9819-4f3a-8648-0096ace6d3fd,2014-06-04T05:19:43Z,2014-06-04T05:34:43Z,92675303-CA5B-136A-169B-E764C5753F06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,410620009,Well child visit (procedure),133.36,269.36,0.0,NaN,None
2,678e03b1-21ac-fbfa-3506-17bee72a6335,2015-06-10T05:19:43Z,2015-06-10T05:34:43Z,92675303-CA5B-136A-169B-E764C5753F06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,NaN,None
3,45987160-8815-858e-893b-6490a63e919c,2015-04-10T03:27:04Z,2015-04-10T03:42:04Z,A0B63E97-B6FD-5FE1-8F2D-2BEC915EFA97,44f0ad91-f33b-3fe5-ba4f-868503d593fe,a79c71d6-e0b4-3450-a0e1-ee08146aeb74,734afbd6-4794-363b-9bc0-6a3981533ed5,wellness,162673000,General examination of patient (procedure),133.36,677.36,0.0,NaN,None


In [20]:
# Assessing data types and null-value counts 
encounters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104262 entries, 0 to 104261
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Id                   104262 non-null  object 
 1   START                104262 non-null  object 
 2   STOP                 104262 non-null  object 
 3   PATIENT              104262 non-null  object 
 4   ORGANIZATION         104262 non-null  object 
 5   PROVIDER             104262 non-null  object 
 6   PAYER                104262 non-null  object 
 7   ENCOUNTERCLASS       104262 non-null  object 
 8   CODE                 104262 non-null  int64  
 9   DESCRIPTION          104262 non-null  object 
 10  BASE_ENCOUNTER_COST  104262 non-null  float64
 11  TOTAL_CLAIM_COST     104262 non-null  float64
 12  PAYER_COVERAGE       104262 non-null  float64
 13  REASONCODE           12557 non-null   float64
 14  REASONDESCRIPTION    12557 non-null   object 
dtypes: float64(4), in

In [21]:
# Evaluating outliers and data distribution of fields
encounters_df.describe()

,CODE,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE
count,1.042620e+05,104262.000000,104262.000000,104262.000000,1.255700e+04
mean,2.408693e+08,123.134884,319.159362,49.636078,2.047645e+08
std,1.366000e+08,21.180931,210.404781,142.918605,1.601317e+07
min,3.387900e+07,68.260000,68.260000,0.000000,2.009360e+08
25%,1.626730e+08,133.360000,249.770000,0.000000,2.009360e+08
50%,1.626730e+08,133.360000,269.360000,0.000000,2.009360e+08
75%,4.106200e+08,133.360000,269.360000,0.000000,2.009360e+08
max,4.106200e+08,133.360000,4345.410000,3073.440000,2.717370e+08


In [22]:
# 'REASON DESCRIPTION' field value check
encounters_df['REASONDESCRIPTION'].value_counts()

REASONDESCRIPTION
Lupus erythematosus    11878
Anemia (disorder)        679
Name: count, dtype: int64

In [23]:
# 'REASON CODE' field value check
encounters_df['REASONCODE'].value_counts()

REASONCODE
200936003.0    11878
271737000.0      679
Name: count, dtype: int64

In [24]:
# Assessing for invalid dates i.e. if STOP date occurs before START date
date_assessment = encounters_df.copy()
date_assessment['START'] = pd.to_datetime(date_assessment['START'])
date_assessment['STOP'] = pd.to_datetime(date_assessment['STOP'])

start_stop_check = date_assessment[date_assessment['STOP'] < date_assessment['START']]
start_stop_check

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION


In [25]:
# Checking for duplicate encounter records

id_count = encounters_df.groupby('Id')['Id'].count().reset_index(name='ID_count')
id_count['ID_count'].value_counts()

ID_count
1    104262
Name: count, dtype: int64

## 3. Symptom data Assessment

In [26]:
# High-level view
symptoms_df.head(4)

,PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS
0,28d7b56c-6056-d0a2-2991-39d6e917216c,NaN,white,hispanic,26,NaN,Lupus erythematosus,4,Rash:34;Joint Pain:39;Fatigue:9;Fever:12
1,6c434506-fb4b-3e3f-c19d-553dec3b6c17,NaN,black,hispanic,58,NaN,Lupus erythematosus,4,Rash:19;Joint Pain:44;Fatigue:48;Fever:15
2,44a8ca45-6c6e-38bb-fac0-ddbf7a7ee3a4,NaN,white,nonhispanic,50,NaN,Lupus erythematosus,4,Rash:2;Joint Pain:32;Fatigue:12;Fever:6
3,780ec78c-22a0-fcdb-17c6-ae9b2fcace9c,NaN,white,hispanic,39,NaN,Lupus erythematosus,4,Rash:30;Joint Pain:30;Fatigue:41;Fever:19


In [27]:
# Assessing data types and null-value counts 
symptoms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PATIENT       1423 non-null   object 
 1   GENDER        0 non-null      float64
 2   RACE          1423 non-null   object 
 3   ETHNICITY     1423 non-null   object 
 4   AGE_BEGIN     1423 non-null   int64  
 5   AGE_END       0 non-null      float64
 6   PATHOLOGY     1423 non-null   object 
 7   NUM_SYMPTOMS  1423 non-null   int64  
 8   SYMPTOMS      1423 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 100.2+ KB


In [28]:
# Evaluating outliers and data distribution of fields
symptoms_df.describe()

,GENDER,AGE_BEGIN,AGE_END,NUM_SYMPTOMS
count,0.0,1423.000000,0.0,1423.000000
mean,NaN,45.673226,NaN,3.859452
std,NaN,13.345750,NaN,0.347677
min,NaN,25.000000,NaN,3.000000
25%,NaN,34.000000,NaN,4.000000
50%,NaN,44.000000,NaN,4.000000
75%,NaN,55.000000,NaN,4.000000
max,NaN,79.000000,NaN,4.000000


In [29]:
# Checking for duplicate 'Patient Id' records
id_count = symptoms_df.groupby('PATIENT')['PATIENT'].count().reset_index(name='ID_count')

symptoms_df[symptoms_df['PATIENT'].isin(id_count[id_count['ID_count']>1]['PATIENT'])].head(4)

,PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS
5,ca424f09-df70-f4a9-986f-8e11f4dd838b,NaN,white,nonhispanic,51,NaN,Lupus erythematosus,4,Rash:33;Joint Pain:9;Fatigue:25;Fever:0
6,ca424f09-df70-f4a9-986f-8e11f4dd838b,NaN,white,nonhispanic,51,NaN,Anemia (disorder),3,Rash:33;Joint Pain:9;Fatigue:25;Fever:0
8,99249ff1-59a9-dc6e-c152-4ca393cd57c5,NaN,white,hispanic,35,NaN,Lupus erythematosus,4,Rash:7;Joint Pain:22;Fatigue:3;Fever:9
9,99249ff1-59a9-dc6e-c152-4ca393cd57c5,NaN,white,hispanic,35,NaN,Anemia (disorder),4,Rash:7;Joint Pain:22;Fatigue:3;Fever:9


In [30]:
# Evaluating 'Pathology' type other than 'Lupus'
symptoms_df.groupby('PATHOLOGY')['NUM_SYMPTOMS'].value_counts()

PATHOLOGY            NUM_SYMPTOMS
Anemia (disorder)    4               400
                     3                75
Lupus erythematosus  4               823
                     3               125
Name: count, dtype: int64

In [31]:
# assessing cases for 'Lupus' where symptoms is 3 instead of 4
symptoms_df[(symptoms_df['PATHOLOGY']=='Lupus erythematosus') & (symptoms_df['NUM_SYMPTOMS']==3)]

,PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS
4,cf5956bb-34f2-841b-2505-57b99991c377,NaN,white,nonhispanic,29,NaN,Lupus erythematosus,3,Rash:28;Joint Pain:26;Fatigue:33;Fever:5
17,67773be8-22f0-b24a-04af-37d8b015124a,NaN,white,nonhispanic,60,NaN,Lupus erythematosus,3,Rash:10;Joint Pain:44;Fatigue:45;Fever:18
18,d6cf05d4-cf6b-1ecc-d176-396e53f23827,NaN,white,hispanic,47,NaN,Lupus erythematosus,3,Rash:39;Joint Pain:3;Fatigue:48;Fever:7
30,5ad9f80a-4574-acaf-ec7e-a82651493c5a,NaN,white,hispanic,55,NaN,Lupus erythematosus,3,Rash:22;Joint Pain:3;Fatigue:20;Fever:9
34,6c52c774-934f-40d1-5637-858f2b873f0f,NaN,black,nonhispanic,36,NaN,Lupus erythematosus,3,Rash:39;Joint Pain:36;Fatigue:49;Fever:19
...,...,...,...,...,...,...,...,...,...
1369,906fc06b-90e0-4e96-7f57-72e4aacb14b9,NaN,white,hispanic,65,NaN,Lupus erythematosus,3,Rash:24;Joint Pain:10;Fatigue:34;Fever:19
1380,400ecdea-126b-9ff2-4256-b03b736e23d1,NaN,white,nonhispanic,49,NaN,Lupus erythematosus,3,Rash:18;Joint Pain:46;Fatigue:18;Fever:18
1398,5675b556-132c-835d-87de-efab20d64b54,NaN,black,nonhispanic,38,NaN,Lupus erythematosus,3,Rash:14;Joint Pain:20;Fatigue:30;Fever:5
1404,eb9540c8-eebf-a528-db84-d9ce1269b79a,NaN,white,nonhispanic,42,NaN,Lupus erythematosus,3,Rash:32;Joint Pain:16;Fatigue:39;Fever:18


## 4. Medication data Assessment

In [32]:
# High-level view
medications_df.head(4)

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2020-04-22T02:47:09Z,NaN,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402DFAEB-1888-F562-4651-F75203EB1E28,849574,Naproxen sodium 220 MG Oral Tablet,274.28,0.0,45,12342.60,200936003.0,Lupus erythematosus
1,2020-04-22T02:47:09Z,2020-05-17T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402DFAEB-1888-F562-4651-F75203EB1E28,312615,predniSONE 20 MG Oral Tablet,16.04,0.0,1,16.04,200936003.0,Lupus erythematosus
2,2021-08-01T02:47:09Z,2021-08-15T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,FBC2F3B3-AB93-707E-AD9E-337990345F32,241834,cycloSPORINE modified 100 MG Oral Capsule,473.61,0.0,1,473.61,200936003.0,Lupus erythematosus
3,2021-08-01T02:47:09Z,2021-08-15T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,FBC2F3B3-AB93-707E-AD9E-337990345F32,312615,predniSONE 20 MG Oral Tablet,8.21,0.0,1,8.21,200936003.0,Lupus erythematosus


In [33]:
# Assessing data types and null-value counts 
medications_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22237 entries, 0 to 22236
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   START              22237 non-null  object 
 1   STOP               19018 non-null  object 
 2   PATIENT            22237 non-null  object 
 3   PAYER              22237 non-null  object 
 4   ENCOUNTER          22237 non-null  object 
 5   CODE               22237 non-null  int64  
 6   DESCRIPTION        22237 non-null  object 
 7   BASE_COST          22237 non-null  float64
 8   PAYER_COVERAGE     22237 non-null  float64
 9   DISPENSES          22237 non-null  int64  
 10  TOTALCOST          22237 non-null  float64
 11  REASONCODE         21924 non-null  float64
 12  REASONDESCRIPTION  21924 non-null  object 
dtypes: float64(4), int64(2), object(7)
memory usage: 2.2+ MB


In [34]:
# Evaluating outliers and data distribution of fields
medications_df.describe()

,CODE,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE
count,2.223700e+04,22237.000000,22237.000000,22237.000000,22237.000000,21924.0
mean,3.635626e+05,160.660295,23.040369,31.272114,8582.331193,200936003.0
std,2.386926e+05,153.233418,69.314682,97.261063,28250.956109,0.0
min,2.418340e+05,0.360000,0.000000,1.000000,0.360000,200936003.0
25%,2.418340e+05,16.020000,0.000000,1.000000,16.170000,200936003.0
50%,3.126150e+05,134.570000,0.000000,1.000000,139.320000,200936003.0
75%,3.126150e+05,289.950000,0.000000,1.000000,343.860000,200936003.0
max,2.001499e+06,587.300000,460.200000,935.000000,337447.280000,200936003.0


In [35]:
# Investigating cases where dispenses are high i.e. greater than average dispenses

dispenses = medications_df[medications_df['DISPENSES'] > medications_df['DISPENSES'].mean()]
dispenses.head(4)

# Result: reason for high dispenses is 'start' date is in the far past

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2020-04-22T02:47:09Z,NaN,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402DFAEB-1888-F562-4651-F75203EB1E28,849574,Naproxen sodium 220 MG Oral Tablet,274.28,0.0,45,12342.60,200936003.0,Lupus erythematosus
6,2001-11-07T11:51:07Z,NaN,239ae86a-96db-6211-9042-d3f2850aabb8,0133f751-9229-3cfd-815f-b6d4979bdd6a,10D39D6F-7D98-8D03-8866-0A7CD363890B,849574,Naproxen sodium 220 MG Oral Tablet,294.21,0.0,270,79436.70,200936003.0,Lupus erythematosus
15,2017-11-05T17:20:52Z,NaN,6c434506-fb4b-3e3f-c19d-553dec3b6c17,734afbd6-4794-363b-9bc0-6a3981533ed5,ADD4FA37-CABE-A5AA-F9DA-8117464BAD04,849574,Naproxen sodium 220 MG Oral Tablet,253.50,0.0,75,19012.50,200936003.0,Lupus erythematosus
21,1989-09-16T00:06:39Z,NaN,6872def5-772f-427c-3053-de6e1c71ce0a,b046940f-1664-3047-bca7-dfa76be352a4,D8A684CB-6CE1-5909-59E7-3422469D18D0,849574,NAPROXEN SODIUM 220 MG ORAL TABLET,348.99,0.0,418,145877.82,200936003.0,Lupus erythematosus


In [36]:
# Checking for 'descriptions' other than 'Lupus'
medications_df['REASONDESCRIPTION'].value_counts()

REASONDESCRIPTION
Lupus erythematosus    21924
Name: count, dtype: int64

In [37]:
# Assessing for invalid dates i.e. if STOP date occurs before START date
date_assessment = medications_df.copy()
date_assessment['START'] = pd.to_datetime(date_assessment['START'])
date_assessment['STOP'] = pd.to_datetime(date_assessment['STOP'])

start_stop_check = date_assessment[date_assessment['STOP'] < date_assessment['START']]
start_stop_check

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION


In [38]:
# Evaluation for standardizing 'Description' field
medications_df['DESCRIPTION'].value_counts()

DESCRIPTION
predniSONE 20 MG Oral Tablet                  8721
cycloSPORINE  modified 100 MG Oral Capsule    7975
Naproxen sodium 220 MG Oral Tablet            2405
PREDNISONE 20 MG ORAL TABLET                   865
CYCLOSPORINE  MODIFIED 100 MG ORAL CAPSULE     729
prednisone 20 mg oral tablet                   460
cyclosporine  modified 100 mg oral capsule     390
NAPROXEN SODIUM 220 MG ORAL TABLET             241
Vitamin B12 5 MG/ML Injectable Solution        142
naproxen sodium 220 mg oral tablet             138
ferrous sulfate 325 MG Oral Tablet             127
FERROUS SULFATE 325 MG ORAL TABLET              16
VITAMIN B12 5 MG/ML INJECTABLE SOLUTION         16
vitamin b12 5 mg/ml injectable solution          9
ferrous sulfate 325 mg oral tablet               3
Name: count, dtype: int64

In [39]:
# Investigating 'Description' for records where 'Reason code' is missing
missing_codes = medications_df[medications_df['REASONCODE'].isna()]
missing_codes['DESCRIPTION'].value_counts()

# Result: The listed medicines do not seem to be associated with 'Lupus' encounters

DESCRIPTION
Vitamin B12 5 MG/ML Injectable Solution    142
ferrous sulfate 325 MG Oral Tablet         127
FERROUS SULFATE 325 MG ORAL TABLET          16
VITAMIN B12 5 MG/ML INJECTABLE SOLUTION     16
vitamin b12 5 mg/ml injectable solution      9
ferrous sulfate 325 mg oral tablet           3
Name: count, dtype: int64

## 5. Condition data Assessment

In [40]:
# High-level view
conditions_df.head(4)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2020-04-21,NaN,28D7B56C-6056-D0A2-2991-39D6E917216C,402dfaeb-1888-f562-4651-f75203eb1e28,200936003,Lupus erythematosus
1,2001-11-07,NaN,239AE86A-96DB-6211-9042-D3F2850AABB8,10d39d6f-7d98-8d03-8866-0a7cd363890b,200936003,Lupus erythematosus
2,2001-11-07,NaN,239AE86A-96DB-6211-9042-D3F2850AABB8,10d39d6f-7d98-8d03-8866-0a7cd363890b,271737000,Anemia (disorder)
3,2017-11-05,NaN,6C434506-FB4B-3E3F-C19D-553DEC3B6C17,add4fa37-cabe-a5aa-f9da-8117464bad04,200936003,Lupus erythematosus


In [41]:
# Assessing data types and null-value counts 
conditions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4149 entries, 0 to 4148
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   START        4149 non-null   object 
 1   STOP         0 non-null      float64
 2   PATIENT      4149 non-null   object 
 3   ENCOUNTER    4149 non-null   object 
 4   CODE         4149 non-null   int64  
 5   DESCRIPTION  4149 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 194.6+ KB


In [42]:
# Evaluation for standardizing 'Description' field
conditions_df['DESCRIPTION'].value_counts()

DESCRIPTION
Lupus erythematosus    2176
Anemia (disorder)      1035
lupus erythematosus     332
LUPUS ERYTHEMATOSUS     276
anemia (disorder)       168
ANEMIA (DISORDER)       162
Name: count, dtype: int64

## Data Analysis

In [43]:
master_data = pd.read_csv('master_data.csv', delimiter=',', header=0)

FileNotFoundError: [Errno 2] No such file or directory: 'master_data.csv'

## 1. Number of unique patients

In [ ]:
unique_patients = master_data['patient_id'].nunique()
print("Number of unique Lupus patients:", unique_patients)

## 2. Distinct medications use over time

In [44]:
import pandas as pd
import matplotlib.pyplot as plt

data = master_data.copy()
# Convert dispensing_start_date to datetime
data['dispensing_start_date'] = pd.to_datetime(data['dispensing_start_date'], errors='coerce')

# Medicine description by concatenating the medicine components
data['full_medicine_description'] = data['medication_name'] + " " + data['route'] + " " + data['strength']

# Drop rows with missing dispensing_start_date or medicine description
filtered_data = data[['dispensing_start_date', 'full_medicine_description']].dropna()

NameError: name 'master_data' is not defined

In [45]:
# Group data by 3-year periods
filtered_data['eight_year_period'] = (filtered_data['dispensing_start_date'].dt.year // 8) * 8
medication_counts = filtered_data.groupby('eight_year_period')['full_medicine_description'].nunique()

# Reset index for plotting
medication_counts = medication_counts.reset_index()
medication_counts['eight_year_period'] = medication_counts['eight_year_period'].astype(str)

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(medication_counts['eight_year_period'], medication_counts['full_medicine_description'], marker='o', linestyle='-')
plt.title("Distinct Medications Over Time (8-Year Periods)")
plt.xlabel("8-Year Period Start")
plt.ylabel("Number of Distinct Medications")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


NameError: name 'filtered_data' is not defined

In [46]:
before_2008 = filtered_data[filtered_data['dispensing_start_date'] < '2008-01-01']
after_2008 = filtered_data[filtered_data['dispensing_start_date'] >= '2008-01-01']

meds_before_2008 = set(before_2008['full_medicine_description'].unique())
meds_after_2008 = set(after_2008['full_medicine_description'].unique())

NameError: name 'filtered_data' is not defined

In [47]:
print("Medicines prescribed before 2008" ,meds_before_2008)
print("Medicines prescribed after 2008:" ,meds_after_2008)


NameError: name 'meds_before_2008' is not defined

## 3. Gender-race distribution

In [48]:
distinct_patients = master_data['patient_id'].nunique()

race_gender_distribution = master_data.groupby(['race', 'gender'])['patient_id'].nunique()
race_gender_distribution_percentage = race_gender_distribution / distinct_patients * 100

NameError: name 'master_data' is not defined

In [49]:
race_gender_df = race_gender_distribution_percentage.reset_index(name='percentage')

# Group smaller percentages into 'Other'
threshold = 2.5  
race_gender_df['group'] = race_gender_df['percentage'].apply(lambda x: 'Other' if x < threshold else None)

# Fill the 'group' column with 'race - gender' for non-'Other' entries
race_gender_df['group'] = race_gender_df['group'].fillna(race_gender_df['race'] + " - " + race_gender_df['gender'])

# Aggregate data for the 'Other' category
grouped_df = race_gender_df.groupby('group')['percentage'].sum().reset_index()

# Pie chart
plt.figure(figsize=(10, 8))
colors = sns.color_palette('bright', len(grouped_df))
plt.pie(grouped_df['percentage'], labels=grouped_df['group'],
        autopct='%1.1f%%', startangle=140, colors=colors)
plt.title("Percentage of Patients by Race and Gender (Grouped)", fontsize=14)
plt.show()

NameError: name 'race_gender_distribution_percentage' is not defined

## 4. Symptom Severity

### Note: There are no patients in the dataset who have all 4 symptoms more than or equal to 30

In [50]:
unique_patient_data = master_data.drop_duplicates('patient_id')

symptom_columns = ['SYMPTOM_FATIGUE', 'SYMPTOM_FEVER', 'SYMPTOM_JOINT_PAIN', 'SYMPTOM_RASH']
symptom_stats = unique_patient_data[symptom_columns].describe()

NameError: name 'master_data' is not defined

In [51]:
symptom_stats.loc[['min','25%','50%','mean','max']]

NameError: name 'symptom_stats' is not defined

In [52]:
one_symptom_more_than_30 = unique_patient_data[symptom_columns].apply(lambda row: any(row >= 30), axis=1).sum()
pct_symptom_more_than_30 = round(one_symptom_more_than_30 / len(unique_patient_data)*100, 1)

NameError: name 'unique_patient_data' is not defined

In [ ]:
print("Percentage of patients reporting atleast one symptom equal or more than 30:", pct_symptom_more_than_30)